In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bike_train = pd.read_csv('train.csv')

In [ ]:
bike_train = bike_train.set_index('date_time')
bike_train

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
date_time,,,,,,,,,
2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833
...,...,...,...,...,...,...,...,...,...
2020-06-26,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,96150
2020-06-27,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,107001
2020-06-28,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,98568


In [ ]:
bike_test = pd.read_csv('test.csv')

In [ ]:
bike_test = bike_test.set_index('date_time')
bike_test

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
date_time,,,,,,,,,
2021-04-01,108.833,3.000,0.000,2.900,28.333,11.800,20.667,18.333,116640
2021-04-02,116.717,3.850,0.000,2.662,46.417,12.000,19.000,28.500,110607
2021-04-03,82.669,4.000,0.565,2.165,77.258,8.875,16.368,52.847,14026
2021-04-04,44.123,3.466,0.466,3.747,63.288,6.250,17.368,37.671,94160
2021-04-05,147.791,1.500,0.000,1.560,48.176,7.188,18.684,4.459,110533
...,...,...,...,...,...,...,...,...,...
2021-06-26,177.149,3.980,0.223,1.066,74.628,20.312,28.579,36.486,105813
2021-06-27,138.723,2.777,0.135,1.290,70.236,20.812,29.000,18.378,92869
2021-06-28,111.095,3.338,1.270,1.692,70.338,21.000,28.789,35.946,106378


In [ ]:
bike = pd.concat([bike_train,bike_test])
bike

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
date_time,,,,,,,,,
2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833
...,...,...,...,...,...,...,...,...,...
2021-06-26,177.149,3.980,0.223,1.066,74.628,20.312,28.579,36.486,105813
2021-06-27,138.723,2.777,0.135,1.290,70.236,20.812,29.000,18.378,92869
2021-06-28,111.095,3.338,1.270,1.692,70.338,21.000,28.789,35.946,106378


In [ ]:
# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
bike['temp_diff_info'] = bike['high_temp'] - bike['low_temp']
# 2. "덥고 습한 날씨"를 알기 위한 정보
bike['sweat_info'] = bike['high_temp'] * bike['humidity'] 
# 3. "춥고 바람부는 날씨"를 알기 위한 정보
bike['cold_info'] = bike['low_temp'] * bike['wind_speed'] 

In [ ]:
bike.drop(['wind_direction', 'humidity', 'cold_info'], axis=1, inplace=True)

In [ ]:
bike = bike[['sky_condition', 'precipitation_form', 'wind_speed', 'low_temp', 'high_temp', 'Precipitation_Probability', 'temp_diff_info', 'sweat_info', 'number_of_rentals']]

In [ ]:
train_bike_x = bike[0:273]
train_bike_x.drop(['number_of_rentals'], axis=1, inplace=True)
train_bike_y = bike['number_of_rentals'][0:273]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # 모델 정의
model.fit(train_bike_x, train_bike_y) # 학습


LinearRegression()

In [ ]:
test_bike_x = bike[273:]
test_bike_x.drop(['number_of_rentals'], axis=1, inplace=True)
test_bike_y = bike['number_of_rentals'][273:]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
y_hat = model.predict(test_bike_x) # y 예측

score = NMAE(test_bike_y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.5666997759641244


In [ ]:
import statsmodels.api as sm
x=sm.add_constant(train_bike_x)
model=sm.OLS(train_bike_y,x).fit()
model.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      number_of_rentals   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     25.46
Date:                Wed, 08 Dec 2021   Prob (F-statistic):           1.68e-26
Time:                        05:51:21   Log-Likelihood:                -3108.8
No. Observations:                 273   AIC:                             6234.
Df Residuals:                     265   BIC:                             6263.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                      5.684e+04   1.63e+04      3.495      0.001    2.48e+04    8.89e+04
sky_condition              9546.5094   9297.321      1.027      0.305   -8759.510    2.79e+04
precipitation_form         4.747e+04   4.32e+04      1.098      0.273   -3.76e+04    1.33e+05
wind_speed                -8595.4857   1743.682     -4.930      0.000    -1.2e+04   -5162.252
low_temp                    -82.8811    425.283     -0.195      0.846    -920.245     754.482
high_temp                   166.5044    397.110      0.419      0.675    -615.387     948.396
Precipitation_Probability -1773.5073    953.655     -1.860      0.064   -3651.211     104.197
temp_diff_info              249.3855    540.272      0.462      0.645    -814.386    1313.157
sweat_info                   16.5874      7.235      2.293      0.023       2.342      30.833
==============================================================================
Omnibus:                       55.964   Durbin-Watson:                   0.536
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.973
Skew:                          -0.021   Prob(JB):                      0.00251
Kurtosis:                       1.975   Cond. No.                     1.03e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.16e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""